In [5]:
import os
os.environ["CDF_LIB"] = "~/CDF/lib"
import sys
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import julian
import nrlmsise00

from numpy import floor
from spacepy import pycdf
from tqdm import tqdm
%matplotlib widget

In [7]:
sat = 'CNOFS'
start = dt.datetime(2011,9,1,0,0,0)
stop = dt.datetime(2011,10,1,0,0,0)
no_of_days = int(np.ceil((stop - start)/dt.timedelta(days=1)))

In [8]:
%load_ext julia.magic
%julia using Dates;
%julia using ProgressBars;
%julia using DataFrames;
%julia using SatelliteToolbox;
%julia init_space_indices();

The julia.magic extension is already loaded. To reload it, use:
  %reload_ext julia.magic


In [14]:
filepath = "C:\\Users\\soumy\\OneDrive - The University of Texas at Dallas\\Documents\\MURI_Project_1\\Data\\Temp\\"
# filepath = '/Users/user/OneDrive - The University of Texas at Dallas/Documents/MURI Project/Data/Temp/'
os.chdir(filepath)

filename = sat + '_IVM_' + start.strftime("%Y-%m-%d") + '.csv'
CNOFS_Data = pd.read_csv(filename)
CNOFS_Data.index = pd.DatetimeIndex(CNOFS_Data['Unnamed: 0'])
df1=CNOFS_Data;
df1 = df1.iloc[np.arange(0,len(df1),12)]

In [16]:
filepath = 'C:\\Users\\soumy\\OneDrive - The University of Texas at Dallas\\Documents\\MURI_Project_1\\Data\\OMNI\\';
# filepath = '/Users/user/OneDrive - The University of Texas at Dallas/Documents/MURI_Project_1/Data/OMNI/'
os.chdir(filepath)

filename = 'omni2_solar_indices.txt';
SW_Data_hr = np.loadtxt(filename,dtype = float)
time_array = pd.DatetimeIndex([dt.datetime(int(SW_Data_hr[:,0][i]),1,1,0,0,0) +
               dt.timedelta(days = SW_Data_hr[:,1][i]-1) +
               dt.timedelta(hours = SW_Data_hr[:,2][i]) for i in range(len(SW_Data_hr))])

SW_Data_hr = pd.DataFrame(SW_Data_hr)
SW_Data_hr.columns = ['Year','DOY','Hour','KP index','R(sunspot no)','Dst index','Ap index','F10.7 index','AE','AU','AL','polar cap index','Lyman_alpha']
SW_Data_hr.index = time_array
SW_Data_hr = SW_Data_hr[(SW_Data_hr.index >= dt.datetime(start.year,1,1)) & (SW_Data_hr.index < dt.datetime(start.year+1,1,1))]

In [17]:
filepath = 'C:\\Users\\soumy\\OneDrive - The University of Texas at Dallas\\Documents\\MURI_Project_1\\Data\\OMNI\\';
# filepath = '/Users/user/OneDrive - The University of Texas at Dallas/Documents/MURI_Project_1/Data/OMNI/';
os.chdir(filepath)
filename = 'noaa_radio_flux.csv'
flux = pd.read_csv(filename)
time_array = [dt.datetime(int(str(flux.iloc[i,0])[:4]),int(str(flux.iloc[i,0])[4:6]),int(str(flux.iloc[i,0])[6:])) for i in range(len(flux))]
flux.index = pd.DatetimeIndex(time_array)
flux = flux[(flux.index>=dt.datetime(start.year,1,1))&(flux.index<=dt.datetime(start.year+1,1,1))]

In [18]:
N = len(df1)
inputs = np.zeros((N,6)); AP = np.zeros((N,7));

for i in tqdm(range(N)):
    t = df1.index[i]
    jd = t.to_julian_date()
    inputs[i,0] = jd
    inputs[i,1] = df1['altitude'][i]*1000 # in m
    inputs[i,2] = df1['glat'][i] # in deg
    inputs[i,3] = df1['glon'][i] # in deg
    
    idx = np.where((flux.index==dt.datetime(t.year,t.month,t.day)))[0][0]
    inputs[i,4] = round(flux['f107_observed (solar flux unit (SFU))'][idx-1],1)
    
    idxa = np.where((flux.index >= t - dt.timedelta(days=45)) & (flux.index <= t + dt.timedelta(days=45)))[0]-1
    inputs[i,5] = round(np.mean(flux['f107_observed (solar flux unit (SFU))'][idxa]),1)
    
    idx = np.where((SW_Data_hr['Year']==t.year)&(SW_Data_hr['DOY']==t.timetuple().tm_yday))[0]
    AP[i,0] = round(SW_Data_hr['Ap index'][idx].mean(),1)
    
    idx = np.where((SW_Data_hr['Year']==t.year)&(SW_Data_hr['DOY']==t.timetuple().tm_yday)&(SW_Data_hr['Hour']==t.hour))[0][0]
    AP[i,1] = SW_Data_hr['Ap index'][idx] ## 3 hour Ap index at the time
    AP[i,2] = SW_Data_hr['Ap index'][idx-3]  ## 3 hour Ap index 3 hrs before the time
    AP[i,3] = SW_Data_hr['Ap index'][idx-6]  ## 3 hour Ap index 6 hrs before the time
    AP[i,4] = SW_Data_hr['Ap index'][idx-9]
    AP[i,5]=round(SW_Data_hr['Ap index'][np.arange(idx-33,idx-12+1,3)].mean(),1)
    AP[i,6]= round(SW_Data_hr['Ap index'][np.arange(idx-57,idx-36+1,3)].mean(),1)

100%|██████████| 41861/41861 [01:30<00:00, 461.01it/s]


In [19]:
## calculate earth radius at a given lat
def radius(B):
    B = np.deg2rad(B);
    a = 6.378137 * 10**6;
    b = 6.356752 * 10**6;
    c = ((a**2)*np.cos(B))**2
    d = ((b**2)*np.sin(B))**2
    e = (a*np.cos(B))**2
    f = (b*np.sin(B))**2
    R = np.sqrt((c+d)/(e+f))
    return R
# R_e = [radius(inputs[i,2]) for i in range(len(df1))];
dist= [radius(inputs[i,2]) + inputs[i,1] for i in range(len(df1))]

In [20]:
%julia inputs=$inputs; AP=$AP;N= size(inputs)[1]; Data=[];
# %julia gd = [geocentric_to_geodetic(deg2rad(inputs[i,3]),dist[i]) for i=1:N];
%julia for i in 1:N;push!(Data,nrlmsise00(inputs[i,1],inputs[i,2],deg2rad(inputs[i,3]),deg2rad(inputs[i,4]),inputs[i,6],inputs[i,5],AP[i,:],output_si=true,dversion=false)); end;
%julia Data_MSIS = DataFrame(Data);

In [21]:
rho = %julia Data_MSIS.den_Total
nO2 = %julia Data_MSIS.den_O2
nN2 = %julia Data_MSIS.den_N2
nO =  %julia Data_MSIS.den_O
# nAO = %julia Data_MSIS.den_aO
nAr = %julia Data_MSIS.den_Ar
nN =  %julia Data_MSIS.den_N
nHe = %julia Data_MSIS.den_He
nH =  %julia Data_MSIS.den_H
T_exo=%julia Data_MSIS.T_exo
Tz =  %julia Data_MSIS.T_alt
# gd = %julia gd

In [22]:
Data_MSIS = pd.DataFrame({'rho': rho,
                       'nO2': nO2,
                        'nN2': nN2,
                        'nO' : nO,
                        'nN':nN,
                        'nHe':nHe,
                        'nH':nH,
                          'nAr':nAr,
                        'T_exo':T_exo,
                        'Tz':Tz
                       },index = df1.index)

In [23]:
filepath = 'C:\\Users\\soumy\\OneDrive - The University of Texas at Dallas\\Documents\\MURI_Project_1\\Data\\Temp\\';
# filepath = '/Users/user/OneDrive - The University of Texas at Dallas/Documents/MURI_Project_1/Data/OMNI/';
os.chdir(filepath)
Data_MSIS.to_csv(sat+'_MSIS_jl_'+start.strftime("%Y-%m-%d")+'.csv')

In [24]:
Data_MSIS

,rho,nO2,nN2,nO,nN,nHe,nH,nAr,T_exo,Tz
Unnamed: 0,,,,,,,,,,
2011-09-01 00:00:04.078,1.982584e-13,2.923393e+08,2.851013e+10,6.541216e+12,9.931572e+10,3.099589e+12,1.768838e+11,117963.972152,770.908926,770.906185
2011-09-01 00:01:04.078,1.695809e-13,2.145125e+08,2.163631e+10,5.524528e+12,9.290267e+10,2.920195e+12,1.706182e+11,80349.499684,781.139729,781.136793
2011-09-01 00:02:04.078,1.453382e-13,1.583304e+08,1.650940e+10,4.671540e+12,8.695887e+10,2.739786e+12,1.645647e+11,55225.123029,792.293930,792.290763
2011-09-01 00:03:04.078,1.248682e-13,1.175802e+08,1.267243e+10,3.957459e+12,8.137517e+10,2.561474e+12,1.587448e+11,38322.946273,804.211810,804.208386
2011-09-01 00:04:04.078,1.075847e-13,8.784614e+07,9.787006e+09,3.360215e+12,7.606825e+10,2.387906e+12,1.531707e+11,26853.195658,816.718878,816.715187
...,...,...,...,...,...,...,...,...,...,...
2011-09-30 23:55:31.336,1.149018e-14,2.176350e+05,7.084047e+07,2.021651e+11,5.852022e+09,8.790216e+11,8.714868e+10,23.760208,913.089461,913.089353
2011-09-30 23:56:31.336,1.067565e-14,1.671653e+05,5.574756e+07,1.764908e+11,5.085415e+09,8.616041e+11,8.808196e+10,16.594154,912.142398,912.142320
2011-09-30 23:57:31.336,1.000806e-14,1.313191e+05,4.473947e+07,1.557360e+11,4.443768e+09,8.461727e+11,8.901499e+10,11.935507,911.207758,911.207701
